# Démonstration Avancée : Modélisation Complexe et SQL Analytique

Ce notebook illustre deux concepts avancés du cours :
1. **Les Tables de Pont (Bridge Tables)** : Pour gérer les relations Many-to-Many (ex: un patient a plusieurs diagnostics).
2. **SQL Analytique Avancé** : Fonctions de fenêtrage (`RANK`, `LAG`, `OVER`) pour l'analyse OLAP.

Nous utilisons **SQLite** pour exécuter le SQL directement dans ce notebook.

In [ ]:
import sqlite3
import pandas as pd

# Connexion à une base en mémoire
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

def run_sql(query):
    return pd.read_sql_query(query, conn)

## Partie 1 : Gestion des Relations Many-to-Many (Bridge Table)

**Scénario** : Dans un hôpital, une **consultation** (Fait) peut aboutir à **plusieurs diagnostics** (Dimension).

Modèle :
- `FACT_CONSULTATION` : Mesures (durée, coût).
- `DIM_DIAGNOSTIC` : Liste des maladies.
- `BRIDGE_CONSULTATION_DIAG` : Table de pont reliant une consultation à plusieurs diagnostics.

In [ ]:
# 1. Création des tables
cursor.executescript('''
    CREATE TABLE dim_diagnostic (
        diag_id INTEGER PRIMARY KEY,
        nom_maladie TEXT,
        categorie TEXT
    );

    CREATE TABLE fact_consultation (
        consultation_id INTEGER PRIMARY KEY,
        date_cons TEXT,
        medecin TEXT,
        cout REAL
    );

    -- Table de Pont
    CREATE TABLE bridge_consultation_diag (
        consultation_id INTEGER,
        diag_id INTEGER,
        poids REAL DEFAULT 1.0, -- Utile si on veut pondérer l'impact (ex: diag principal = 0.8, secondaire = 0.2)
        FOREIGN KEY(consultation_id) REFERENCES fact_consultation(consultation_id),
        FOREIGN KEY(diag_id) REFERENCES dim_diagnostic(diag_id)
    );
''')

# 2. Insertion de données
cursor.executescript('''
    -- Diagnostics
    INSERT INTO dim_diagnostic VALUES (1, 'Grippe', 'Infectieux');
    INSERT INTO dim_diagnostic VALUES (2, 'Diabète', 'Chronique');
    INSERT INTO dim_diagnostic VALUES (3, 'Hypertension', 'Cardio');

    -- Consultations
    INSERT INTO fact_consultation VALUES (101, '2023-01-15', 'Dr. House', 50.0);
    INSERT INTO fact_consultation VALUES (102, '2023-01-16', 'Dr. Grey', 75.0);

    -- Pont (Relations Many-to-Many)
    -- Consultation 101 : Grippe seulement
    INSERT INTO bridge_consultation_diag VALUES (101, 1, 1.0);

    -- Consultation 102 : Diabète ET Hypertension (Patient complexe)
    INSERT INTO bridge_consultation_diag VALUES (102, 2, 0.5);
    INSERT INTO bridge_consultation_diag VALUES (102, 3, 0.5);
''')

print("Données chargées.")

### Analyse via la Bridge Table

**Question** : Quel est le coût total des consultations par catégorie de diagnostic ?

*Note : Si on somme simplement, la consultation 102 risque d'être comptée deux fois (une fois pour Diabète, une fois pour Hypertension). C'est le piège des relations M:N.*

In [ ]:
query_bridge = '''
SELECT 
    d.categorie,
    COUNT(DISTINCT f.consultation_id) as nb_consultations,
    SUM(f.cout) as cout_total_brut, -- Attention : double compte la consultation 102 !
    SUM(f.cout * b.poids) as cout_pondere -- Correct : utilise le poids pour répartir le coût
FROM fact_consultation f
JOIN bridge_consultation_diag b ON f.consultation_id = b.consultation_id
JOIN dim_diagnostic d ON b.diag_id = d.diag_id
GROUP BY d.categorie
'''

run_sql(query_bridge)

**Interprétation** :
- `cout_total_brut` surestime les coûts car la consultation à 75€ est comptée pour 'Chronique' et pour 'Cardio'.
- `cout_pondere` répartit correctement les 75€ (37.5€ chacun) grâce à la colonne de pondération dans la table de pont.

## Partie 2 : SQL Analytique Avancé (Window Functions)

**Scénario** : Analyse des ventes avec calcul de croissance (Year-over-Year) et classement (Ranking).

In [ ]:
# Préparation des données de vente
cursor.executescript('''
    CREATE TABLE fact_ventes (
        mois TEXT,
        produit TEXT,
        ventes REAL
    );

    INSERT INTO fact_ventes VALUES ('2023-01', 'Ordi', 10000);
    INSERT INTO fact_ventes VALUES ('2023-02', 'Ordi', 12000);
    INSERT INTO fact_ventes VALUES ('2023-03', 'Ordi', 11000);
    
    INSERT INTO fact_ventes VALUES ('2023-01', 'Tablette', 5000);
    INSERT INTO fact_ventes VALUES ('2023-02', 'Tablette', 5500);
    INSERT INTO fact_ventes VALUES ('2023-03', 'Tablette', 4800);
''')

### 1. Analyse de la croissance (LAG)
Calculer l'évolution des ventes par rapport au mois précédent pour chaque produit.

In [ ]:
query_growth = '''
SELECT 
    produit,
    mois,
    ventes,
    LAG(ventes, 1, 0) OVER (PARTITION BY produit ORDER BY mois) as ventes_mois_prec,
    (ventes - LAG(ventes, 1, 0) OVER (PARTITION BY produit ORDER BY mois)) as evolution
FROM fact_ventes
ORDER BY produit, mois
'''

run_sql(query_growth)

### 2. Classement et Part de Marché (RANK, SUM OVER)
Classer les produits par ventes mensuelles et calculer leur part dans le CA total du mois.

In [ ]:
query_rank = '''
SELECT 
    mois,
    produit,
    ventes,
    RANK() OVER (PARTITION BY mois ORDER BY ventes DESC) as classement,
    SUM(ventes) OVER (PARTITION BY mois) as total_mois,
    ROUND(100.0 * ventes / SUM(ventes) OVER (PARTITION BY mois), 2) as part_de_marche_pct
FROM fact_ventes
ORDER BY mois, classement
'''

run_sql(query_rank)